In [54]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ipaddress
from datetime import datetime
import time

In [55]:
data = pd.read_csv("./data_full.csv", sep = ";", error_bad_lines=False)

b'Skipping line 10780784: expected 10 fields, saw 11\nSkipping line 10793876: expected 10 fields, saw 11\n'
b'Skipping line 10821923: expected 10 fields, saw 11\n'
C:\Users\ASUS\Anaconda3\envs\tensorflow-gpu\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [56]:
catalog = pd.read_csv("./catalog.csv", sep = ";", error_bad_lines=False)

In [12]:
data.head()

,timestamp,ip,track_id,cookie,live,user_id,referer,uagent,rightholder,author_id
0,1517778000,230.59.74.120,10912114,42536dc7a8578b0cfac05f704977429a,0,NaN,https%3A%2F%2Frutube.ru%2Fvideo%2F0f5c9c5839b1...,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,2,1480930
1,1517778000,218.222.225.36,10885813,8f92f3f2b1e7a2498761b8cb3b1d03c3,0,NaN,https%3A%2F%2Frutube.ru%2Fvideo%2Fd8068436dcf5...,Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.3...,2,301323
2,1517778000,55.63.241.224,6948236,336d5ebc5436534e61d16e63ddfca327,-,2.231332e+09,http%3A%2F%2Fandroid.rutube.ru%2F,okhttp/2.6.0,-,-
3,1517778000,230.34.98.99,8481402,cf8dd809edf104873a1c57921b34f25e,0,NaN,https%3A%2F%2Frutube.ru%2Fvideo%2F1058cf249e46...,Mozilla/5.0 (Linux; Android 7.1.2; Redmi 4X Bu...,NaN,721346
4,1517778000,232.3.41.226,10908060,4ca525a1d60de6d74e3c32529fe47ab5,0,NaN,https%3A%2F%2Fyastatic.net%2Fvideo-player%2F0x...,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,135,599848


In [13]:
catalog.head()

,idec,track_id,video_id,series_id,title,season,episode,episode_global,date_efir,duration
0,PR592505,10258734,cd59084eb50e6ebe9fe20ad04e0c17db,1.045685e+10,"\tБыть или не быть, 1 сезон, 1 серия",1.0,1.0,1.0,02.04.2017 22:00:34,NaN
1,PR609630,10843255,a5d2e6bae4752d9ec0fae9b53f00700d,1.059247e+10,NaN,11.0,1.0,1.0,NaN,NaN
2,PR597128,10573153,bcd45606c1f7cbb5cb9d1dbd7d346cc3,1.047956e+10,NaN,7.0,1.0,1.0,06.09.2017 22:46:05,NaN
3,PR257331,6617650,609217ac7b8c4e55d44c4a29bf52554b,NaN,"""Интерны. История болезни""",NaN,NaN,NaN,12.06.2012 22:00:44,NaN
4,PR529698,7991493,bde3620ba038b3e8dd41d03bf516a7eb,1.024102e+10,#ЖАННАПОЖЕНИ,1.0,1.0,101.0,05.09.2015 13:30:00,NaN


In [14]:
def add_device(df):
    df['device'] = pd.Series('0', index=df.index)

    for i, row in df.iterrows():
        if df['uagent'].iloc[i].find('Windows') != -1 or df['uagent'].iloc[i].find('Linux') != -1 or df['uagent'].iloc[i].find('Mozilla') != -1:
            df['device'][i] = 'desktop'
        if df['uagent'].iloc[i].find('iPhone') != -1 or df['uagent'].iloc[i].find('RutubeAndroid') != -1 or df['uagent'].iloc[i].find('okhttp') != -1 or df['uagent'].iloc[i].find('CFNetwork') != -1:
            df['device'][i] = 'mobile'
        if df['uagent'].iloc[i].find('iPad') != -1:
            df['device'][i] = 'tablet'
            
    return df

In [60]:
def mergeData(data1, data2, field):
    data1[field] = pd.to_numeric(data1[field], errors = 'coerce')
    data2[field] = pd.to_numeric(data2[field], errors = 'coerce')
    return pd.merge(data1, data2, left_on=field, right_on=field)

In [16]:
def convIpToNumber(data):
    return data.apply(lambda x: int(ipaddress.IPv4Address(x)))

In [19]:
def applyDate(x):
    if (x == '00.00.0000 0:00:00' or x == None or x ):
        return 1e-6
    data = time.strptime(x, "%d.%m.%Y %H:%M:%S")
    return int(time.mktime(data))

In [20]:
def confDateStringToTimestamp(data):
    return data.apply(lambda x: applyDate(x))

In [21]:
def filter_having_more_than_n_views(data, user_column = "ip", seq_len = 4):
    by_user = data.groupby(user_column)
    by_user = by_user.filter(lambda x: len(x) >= seq_len)
    return by_user

def extract_sequences_by_user(data, user_column = "ip"):
    return data.groupby(user_column)

In [61]:
def prepare_data(data, catalog):
    
    data = mergeData(data, catalog, "track_id")
    
    data['ip'] = convIpToNumber(data['ip'])
    
    data = filter_having_more_than_n_views(data)
    
    data = data.drop(['live', 'user_id', 'rightholder'], axis=1)
    
    data['date_efir'] = confDateStringToTimestamp(data['date_efir'])
    
    data = add_device(data)
    
    return data

data_prepared = prepare_data(data.head(100000), catalog)
data_prepared.head()

C:\Users\ASUS\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


TypeError: strptime() argument 0 must be str, not <class 'float'>

In [ ]:
def vocab_size(catalog):
    return catalog["track_id"].unique().shape
print(vocab_size(catalog))

In [ ]:
def train_generator(seq_by_user, x_len = 3):
    for ip, group in seq_by_user:
        group = group.sort_values("timestamp").values
        x = group[0:x_len]
        y = group[x_len]
        yield (x, y)
        
print(next(train_generator(extract_sequences_by_user(data_prepared[0:10000]))))